In [1]:
import pandas as pd
import numpy as np
import pickle

from utils.Sharp import SharpPortfolio
from utils.Markov import MarkovPortfolio
from utils.Tobin import TobinPortfolio

# Disable jedi autocompleter
%config Completer.use_jedi = False

In [2]:
df = pd.read_csv('results/prediction_table.csv', index_col=0)
pct = pd.read_csv('data/ticker_data_preprocessed.csv', index_col=0)
pct = pct.drop(['sector', '2022-07-29', '2022-07-28', '2022-07-27', '2022-07-26', '2022-07-25'], axis=1)
sp500 = pd.read_csv('data/sp500.csv', index_col=0)['Изм. %'][5:]
bonds = pd.read_csv('data/bonds.csv', index_col=0)['Изм. %'][5:]

In [3]:
sp500 = sp500.apply(lambda x: float(x[:-1].replace(',', '.')) / 100)
bonds = bonds.apply(lambda x: float(x[:-1].replace(',', '.')) / 100)

# Обработка даты 

def month(dates : list):
    new_dates = []
    for d in dates:
        date = d.split()
        months = {'янв.' : '01', 'февр.': '02', 'мар.': '03', 'апр.': '04', 'мая': '05',
                  'июн.' : '06', 'июл.': '07', 'авг.': '08', 'сент.': '09', 'окт.': '10', 'нояб.': '11', 'дек.': '12'}
        date = date[0]+'-'+months[date[1]]+'-'+date[2]
        new_dates.append(date)
    return new_dates

sp500.index = month(sp500.index)

## Создаю словари для каждого метода кластеризации

In [4]:
cols = df.columns[1:]

**общий словарь**: ключ - метод кластеризации, значение - словарь(*ключ - номер кластера, значения - тикеры*)

In [5]:
all_accets = {}
for col in cols:
    accets = {}
    for i in set(df[col].unique()):
        accets[i] = list(df.loc[df[col] == i].index.values)
    all_accets[col] = accets

## Выберу из каждого кластера 2 лучшие акции по значению Шарпа и создам портфель для каждого метода кластеризации

In [6]:
def choose_best_accets(dict_accets, df_pct):
    accets_to_port = []
    for cluster in dict_accets.values():
        if len(cluster)==1:
            accets_to_port.append(cluster[0])
        else:
            dict_with_sharp = {}
            sharp = 0
            first_sharp = -100
            second_sharp = -200
            # считаем Шарпа для каждой акции
            for ticker in cluster:
                tickers_pct = df_pct.loc[df_pct.index == ticker].values
                sharp = (tickers_pct.mean()-0.03)/tickers_pct.std()
                dict_with_sharp[sharp] = ticker

            # выбираем 2 лучших акции
            for sharp in dict_with_sharp.keys():
                if first_sharp < sharp:
                    second_sharp = first_sharp
                    first_sharp = sharp
                elif first_sharp > sharp and second_sharp < sharp:
                    second_sharp = sharp
            accets_to_port.append(dict_with_sharp[first_sharp])
            accets_to_port.append(dict_with_sharp[second_sharp])
    return accets_to_port    

In [7]:
# Сохраняю расчеты в словарь: ключ - название портфеля(метод кластеризации), значение - тикеры
ports = {}
for col in cols:
    ports[col] = choose_best_accets(all_accets[col], pct)

## Найду лучшие методы кластеризации исходя из доходности

Веса в портфеле определяются значением коэффициента Шарпа

In [8]:
bonds.index = pd.to_datetime(bonds.index)
sp500.index = pd.to_datetime(sp500.index)

# Настройка размерности

sp500 = sp500.drop(list(set(sp500.index) - set(bonds.index)))
bonds = bonds.drop(list(set(bonds.index) - set(sp500.index)))

In [9]:
pct_monthly = {}
sp500_monthly = {}
bonds_monthly = {}

pct = pct.T[:-1]
pct.index = pd.to_datetime(pct.index)

In [10]:
# Настройка размерности

pct = pct.drop(list(set(pct.index) - set(bonds.index)))
bonds = bonds.drop(list(set(bonds.index) - set(pct.index)))
sp500 = sp500.drop(list(set(sp500.index) - set(pct.index)))

In [11]:
dates = list((sp500.reset_index().groupby(pd.Grouper(key="index", freq="M")).std()).index)

In [12]:
last_date = dates[0]
pct_monthly[last_date] = pct[(pct.index<=last_date)]
sp500_monthly[last_date] = sp500[(sp500.index<=last_date)]
bonds_monthly[last_date] = bonds[(bonds.index<=last_date)]
for date in dates[1:]:
    pct_monthly[date] = pct[((pct.index>last_date)&(pct.index<=date))]
    sp500_monthly[date] = sp500[((sp500.index>last_date)&(sp500.index<=date))]
    bonds_monthly[date] = bonds[((bonds.index>last_date)&(bonds.index<=date))]
    last_date=date

Для каждого портфеля создаю словарь с доходностями

In [13]:
def create_dict(dates, pct_monthly, tickers):
    d = {}
    for date in dates:
        d[date] = pct_monthly[date][tickers]
    return d

In [14]:
port_names = list(ports.keys())
portfolios = {}
for port in port_names:
    portfolios[port] = create_dict(dates, pct_monthly, ports[port])

В переменной portfolios хранятся словари(каждый метод кластеризации) с доходностями своих тикеров

### Расчет портфелей

1) Выбираем метод кластеризацци
2) Считаем для него оптимизацию

In [15]:
def calc_ports(test, riskless_ret, index, mu, sigma):
    sharp_values, markov_values, tobin_values, traynor_values, qsharp_values = [], [], [], [], []
    # Расчет Шарпа
    sharp = SharpPortfolio(mu, sigma, riskless_ret.mean())
    sh_values = test @ sharp.fit()
    for value in sh_values:
        sharp_values.append(value)

    # Расчет Марковица
    markov = MarkovPortfolio(mu, sigma, index)
    m_values = test @ markov.fit()
    for value in m_values:
        markov_values.append(value)

    # Расчет Тобина

    tobin = TobinPortfolio(mu, sigma, riskless_ret.mean(), index.std())
    #
    tobin.fit()
    #
    tob_values = test @ tobin.fit()
    for value in tob_values:
        tobin_values.append(value)


    df_all_ports = pd.DataFrame({'date':test.index,
                                 'sharp':sharp_values,
                                 'markov':markov_values,
                                 'tobin':tobin_values})
    return df_all_ports

In [16]:
dict_ports = {}
for port in list(portfolios.keys()):
    cluster = portfolios[port]
    test = cluster[dates[0]]
    riskless_ret = bonds_monthly[dates[0]]
    index = sp500_monthly[dates[0]]
    mu = test.mean().values
    sigma = test.cov().values

    dict_ports[port] = calc_ports(test, riskless_ret, index, mu, sigma)
    for date in dates:
        test = cluster[date]
        riskless_ret = bonds_monthly[date]
        index = sp500_monthly[date]
        mu = test.mean().values
        sigma = test.cov().values
        layer = calc_ports(test, riskless_ret, index, mu, sigma)
        dict_ports[port] = pd.concat([dict_ports[port], layer])

-0.28314520607563415 0.0994018349506062
-0.28314520607563415 0.0994018349506062
-0.45701646965062925 -0.14150455945517845
-0.1463378617966716 -0.0034212318466746135
-0.1388560274425439 0.3012456065074133
0.5020982185416988 0.38954057149917404
0.05686678571529653 0.059927211087331705
-0.2629340916234901 0.04476488034627375
0.2453173663906723 0.2964507535014571
-0.17080787861090632 0.062286190965803116
-0.2886027992932012 -0.27241378056650395
0.1977631278956493 0.194331874525548
-0.2252440135281267 -0.2868097121601288
0.7891955560478493 0.5264945652060085
0.3598102195823175 0.07745840905392956
0.5033058521560935 -0.23767371300071705
0.2483294340513214 0.1794274968050965
-0.01042110532413169 -0.2833856027187096
0.6992975699025024 0.22067635942500607
0.06453440547334881 0.04754861381473849
-0.03642432416189589 -0.02276160055973706
0.3408035988424436 0.16401082093505776
0.5494758558896634 0.1091174953552649
0.1013406415368636 0.2815230305673494
0.25673597691686606 0.501934269613846
0.236103

-0.5784716858627476 -0.0172490333961369
1.4533332449929475 0.13803766925881616
1.2159669180261623 0.2676512437665853
-2.108472054249326 0.3812516934076361
-0.08687302526381627 -0.07414753525494466
0.781901057442123 0.08659029654379623
-1.2462448496717031 0.018607363675428378
-1.5339289456415828 0.4520391655156358
-0.28726982220507247 -0.06240225281996468
-0.8469980491201213 -0.005474395878382262
-1.3959542683484842 -0.11270066390584611
-0.26810418187685114 -0.041505512637006046
-1.0280473132756647 0.20012197884874758
-0.19710310981563162 -0.024185954159591114
-0.07709980600047672 0.12847260219473286
-0.6413626851869649 -0.21147480766653537
0.6227785357852867 0.7673240392346348
-0.003306164309854467 0.4256633956681324
-0.003306164309854467 0.4256633956681324
-0.5295157146604242 -0.2526658175559337
-0.3949236458081287 -0.24687945269441308
-0.18579934406145288 0.22283800183801822
0.3274967866713431 0.23955698390677588
0.13044614216536052 0.12793814580137292
-0.3396319048004717 0.029215052

0.20862235334451984 0.10015609323696748
0.37083025114575896 0.12300999519358766
0.025466934423974782 0.14866099760118573
0.3411866883997124 0.4901231662155794
0.3690581168637614 0.1189620896721912
1.015248589023428 0.0018073266071877599
0.06659953414201972 -0.21095629286431555
1.1777144426557975 0.5216481678676768
0.03535325885308546 0.34419533307132527
0.301627662391866 0.16022353429809952
0.6357317685755177 0.23378902425805026
-0.0001074639194894792 0.6577561557724486
0.7866645424226498 0.13488864315605695
0.17832627891468952 -0.057823454327995794
1.5651739276389438 0.6005109709105668
0.03582255919044463 0.47023413068111664
0.5052765421374769 0.045145842876303244
-0.6374679307516609 0.10667605586652786
-0.4398624551531445 -0.038934171997677645
1.346217202531295 0.12279031859307742
0.7872578229049073 0.15078651078784225
-1.568768150205292 0.4613791258306767
-0.16867587097430034 -0.16929697789509238
0.7356441578105868 0.22728678641061423
-1.4000810068172183 -0.11254679290939208
-0.9875

-0.3430097054894506 -0.1882200958860104
-0.21522601487361226 0.22218208338364465
0.31629162823371154 0.21852391209787758
0.2528779957995222 0.2659160701405481
-0.37603501049619187 -0.11474924341935987
-0.07732896414573548 0.0021781440035637677
-0.2589707973345549 -0.05484618188766786
-0.22381448377898824 -0.17272859854000547
0.10784831740016514 0.11173587474227707
-0.20959386423733406 -0.27490321496101455
0.816503218659339 0.5032801170701783
0.23965255788636622 0.040539189886714115
0.5427909055568229 -0.25508055424366466
-0.1000497384971389 -0.14103251957727447
-0.027722208118316737 -0.33881365807242614
0.6862567251974931 0.24097625357683927
0.2068486013261306 0.1924576789514654
-0.15987760454035527 -0.14153511206133348
0.2540720996712268 0.10732373978097536
0.4600027626539472 0.09222672758143872
0.06479243524864284 0.2430436096512284
0.1693738185419808 0.3923305932711753
0.542640743586654 0.17158532503276433
1.2242045935580523 -0.13074912752511264
0.09950871240098962 -0.21782395417295

-2.3259571618143986 0.2909333679727981
-0.06586019583071787 -0.049534220670436044
0.7595245617337915 0.06730447067684811
-1.4704945646518723 -0.011895099560516202
-1.9059628545197917 0.4213996633536703
-0.2792045925343457 -0.051186917721951365
-0.9034352208371605 0.04541647922431791
-1.6818134187259168 -0.15770178881899638
-0.2721162905402144 -0.003879572347393835
-1.2219983192516575 0.2403282838077914
-0.20734424234465362 -0.04744443690826992
-0.05239261599654634 0.17401969284827914
-0.6479308684390052 -0.17726290601459174
0.6083763632840127 0.7811570287794787
-0.22275813262664737 0.15010844174306345
-0.22275813262664737 0.15010844174306345
-0.51945270467787 -0.20029537984575235
-0.21577231797459145 -0.006606969577683218
-0.18465256738956243 0.2700500595252598
0.47840214217834237 0.3898523451904342
0.26501891628628776 0.2595191302328767
-0.28883758200203247 0.006476241640774383
0.060371581363088074 0.12859560410078036
-0.3069202760061205 -0.08959322786372759
-0.26326302908997695 -0.20

...

0.6929645977247225 0.12046290305095135
-1.1022614872373546 0.08598325834004308
-1.8890763571096132 0.5827109856874043
-0.23810186370168204 -0.04690619377916316
-0.7022818122682003 -0.0034512313359035354
-1.3293127980367125 -0.10557698034613645
-0.16972217426382838 0.043640278850809976
-0.8741265919550288 0.21698772171965724
-0.28099283009031706 -0.10975981278416756
-0.00806295026455163 0.16508795269189036
-0.5241642296002131 -0.17724410974181537
0.43497595008351553 0.5501079429328929
-0.08010972649565754 0.4193562037588063
-0.08010972649565754 0.4193562037588063
-0.5879051201236881 -0.23263763499876705
-0.2342980385916389 -0.0788546902782199
-0.24595428590223653 0.14727871522286026
0.527525774476316 0.43553483846169366
0.04769978967296942 0.06052611467126494
-0.26914116796968224 0.06740325773271434
0.10980892384708658 0.15633785295981256
-0.39770316006729545 -0.1945367025901953
-0.19489334073577524 -0.159368116277088
0.15590880222772063 0.15993275469435905
-0.1771647526304895 -0.245562

1.0637127388372385 -0.2228071762945241
0.06081959077387156 -0.2320365754375773
0.9825188009523957 0.3769754183052856
0.04333598834492204 0.2502347545541863
0.2355402407419383 0.1179062000534297
0.5179758053365144 0.15393086608691223
-0.13266430798787376 0.5420408931591684
0.8744606448230063 0.06488330659374451
0.2708129861424899 0.017944498120485215
1.2943036478625698 0.5293525437316431
-0.19298106251433791 0.2995732092279408
0.534029697682286 0.03434890615277602
-0.49293582311459505 0.45457795936009054
-0.5413920512851208 -0.0039226816184558914
1.380200807932109 0.1420470163261728
1.067854654278926 0.2558040030880463
-1.8130382523558874 0.09100390976091553
-0.10594345750372194 -0.09165556757166993
0.6869055192019228 0.06265509239367002
-1.1314067452746328 0.0621794205613104
-1.69574117772758 0.21161849667516158
-0.2933427218326674 -0.08943071421609297
-0.8334486079974079 -0.03124865099753878
-1.4776536820430628 -0.10282834971034825
-0.2991435040147826 -0.08724442313484808
-0.848278425

-0.2330186636866329 0.050426962811317304
-0.09624187908922154 -0.036909972051324624
-0.06247335562114345 0.12717786071751566
-0.263675724054138 -0.22679058737322705
0.19144485889799562 0.1948711145859627
-0.25456741730419946 -0.30823960472343415
0.6991543188580418 0.487395653735825
0.38819118031806604 0.2039312253741699
0.3889409777186511 -0.27705609433126693
0.11164386576041471 0.09227255821961294
-0.07828508479637164 -0.3096642836404245
0.5678639900103052 0.18969414856485248
0.013445073198217977 0.010200489532177007
-0.17919392765380437 -0.17817973401564258
0.343292457858913 0.2126912717264406
0.4228982744248233 0.11541004877357028
-0.02335251477935935 0.1494236588270976
0.31132703670822953 0.4687072337653236
0.20094518094577307 -0.07306925935654782
0.9088285499009694 -0.1987264193853924
0.03497494351599656 -0.23248790612851497
1.0529118651356255 0.4399701011911212
0.027501211672533242 0.2462217254256666
0.21514800267600484 0.11283537676323672
0.42804318327123014 0.11712047594768961


-0.3212729941154797 -0.05496103348518001
-0.3612921417878072 -0.21510005383323053
-0.17586872447519392 0.20741407954662092
0.24977745461179748 0.18763094833883914
0.247786476087007 0.25575467008010255
-0.3535358601161026 -0.06271540884419226
-0.04970852689368011 -0.007758186692876112
-0.012338325853352851 0.15752763417599175
-0.2586464288531515 -0.22612728741158442
0.17628491019307635 0.17228343115238692
-0.232300076821276 -0.29601334612092506
0.7260178970550931 0.5055571695941158
0.4848922078475101 0.2609274610917944
0.41824932075859433 -0.20786924367645837
-0.039043527979542095 -0.05922289938119668
0.00041709079958100276 -0.2057278273140258
0.39024770083185467 0.07091116572419402
0.10687150565178714 0.11064342389266846
-0.16410623838387475 -0.15036465200748017
0.28208845997677817 0.16822645357992808
0.24494399406660866 -0.02956135042891853
0.05866147727955107 0.19727224509301614
0.3752313717376094 0.5249598437242955
0.35629520128465564 0.10578623195384952
0.9168301447742294 -0.062015

-0.564328209849506 -0.0028131172006299705
1.5326112212813798 0.13503144579921975
1.0733825886494321 0.23617634254976164
-1.6063639586464145 0.2100198711732316
-0.12210747243523103 -0.10704530776687071
0.6656886761581122 0.0742756112806758
-1.1374075820989544 0.06955661681863355
-2.0203048072034426 0.26746550618666104
-0.4066881269360414 -0.20402752192630774
-0.7762627326412659 0.03349952603747082
-1.4784879462279594 -0.07783786599393831
-0.2952723853646141 -0.03453280129764097
-1.0087488740798123 0.25447185285590185
-0.2598234176412504 -0.08233268051400566
-0.08010039733417006 0.12660678437610556
-0.5809440794133208 -0.20520036559366955
0.4830553740994637 0.6029644438657297
-0.28314520607563415 0.0994018349506062
-0.28314520607563415 0.0994018349506062
-0.45701646965062925 -0.14150455945517845
-0.1463378617966716 -0.0034212318466746135
-0.1388560274425439 0.3012456065074133
0.5020982185416988 0.38954057149917404
0.05686678571529653 0.059927211087331705
-0.2629340916234901 0.04476488034

In [17]:
df_res_sharp = pd.DataFrame([])
df_res_markov = pd.DataFrame([])
df_res_tobin = pd.DataFrame([])
columns = []

for i, (k, v) in enumerate(dict_ports.items()):
    col_names = v.columns
    df_loc_sharp = pd.DataFrame(data=v[col_names[1]].values, index=v[col_names[0]].values, columns = [k])
    df_res_sharp = pd.concat([df_res_sharp, df_loc_sharp], axis=1)
    df_loc_markov = pd.DataFrame(data=v[col_names[2]].values, index=v[col_names[0]].values, columns = [k])
    df_res_markov = pd.concat([df_res_markov, df_loc_markov], axis=1)
    df_loc_tobin = pd.DataFrame(data=v[col_names[3]].values, index=v[col_names[0]].values, columns = [k])
    df_res_tobin = pd.concat([df_res_tobin, df_loc_tobin], axis=1)
    columns.append(columns)

In [18]:
df_res_sharp['sp500']=sp500
df_res_markov['sp500']=sp500
df_res_tobin['sp500']=sp500

df_res_sharp.to_csv('results/df_res_sharp.csv')
df_res_markov.to_csv('results/df_res_markov.csv')
df_res_tobin.to_csv('results/df_res_tobin.csv')

In [19]:
df_real_sharp = (df_res_sharp + 1).cumprod()
df_real_markov = (df_res_markov + 1).cumprod()
df_real_tobin = (df_res_tobin + 1).cumprod()

df_real_sharp.to_csv('results/df_real_sharp.csv')
df_real_markov.to_csv('results/df_real_markov.csv')
df_real_tobin.to_csv('results/df_real_tobin.csv')

## Метрики Sharp

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import dates as mdates

from scipy.stats import norm

import functools

## Поиск метрик

In [21]:
df_all_ports_pct = pd.read_csv('results/df_res_sharp.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [22]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей

## Безрисковый актив

In [23]:
df_b = pd.read_csv(r'data/bonds.csv', decimal=",")
df_bond = pd.DataFrame()

# Найти среднюю цену за месяц и поделить её на 100
df_bond['date'] = df_b['Дата']
df_bond['цена'] = df_b['Цена']/100
df_bond.date = pd.to_datetime(df_bond.date)
df_bond.set_index('date', inplace = True)

df_bond = df_bond.mean()

## Риск портфеля

In [24]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [25]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [26]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta_ports = (cov_ports['sp500']/var_ports['sp500']).to_frame().T.rename({'sp500':'Beta'})

## Коэффициент VaR

In [27]:
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [28]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [29]:
var = l_pr_values*(1+norm.ppf(0.001, mean_for_var, std_for_var))/l_pr_values-1
var = pd.DataFrame(data=var.reshape(1,-1), columns=mean_for_var.index, index=['VaR'])
var

,original_n,deco_pca_Kmean,deco_pca_Agglo,deco_pca_MiniB,deco_pca_Gauss,deco_pca_Spect,deco_fast_Kmean,deco_fast_Agglo,deco_fast_MiniB,deco_fast_Gauss,...,neur_lstm_Gauss,neur_lstm_Spect,orig__Kmean,orig__Agglo,orig__MiniB,orig__Gauss,orig__Spect,orig__origi,orig__rando,sp500
VaR,-0.069293,-0.06964,-0.082701,-0.065872,-0.068442,-0.072928,-0.070099,-0.0783,-0.07192,-0.072862,...,-0.122632,-0.083559,-0.066168,-0.087908,-0.061946,-0.063723,-0.073511,-0.069293,-0.07907,-0.108396


## Коэффициент Шарпа

In [30]:
std_ports = df_all_ports_pct.std()

sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [31]:
def find_max_drawdown(prices):

    max_price = prices.iloc[0]
    curr_drawdown = 0
    max_drawdown = 0
    curr_left = 0
    left = 0
    right = 0
    for i in range(0, len(prices)):
        curr_drawdown = (prices.iloc[i] / max_price - 1) * 100
        # print(type(curr_drawdown))
        if curr_drawdown < max_drawdown:
            max_drawdown = curr_drawdown
            left = curr_left
            right = i
        if prices.iloc[i] > max_price:
            max_price = prices.iloc[i]
            curr_left = i
    return max_drawdown

In [32]:
def find_max_recovery(prices):
    
    def calc_growth(prices):

        growth = []
        past_p = 0
        for p in prices:
            if past_p:
                growth.append(p - past_p)
            past_p = p
        return growth

    growth = calc_growth(prices)
    s = 0
    left = 0
    right = 0
    curr_left = 0
    max_recovery = 0
    for i in range(0, len(growth)):
        if not s:
            curr_left = i
        s += growth[i]
        if s > 0:
            s = 0
            if max_recovery < (i - curr_left):
                max_recovery = i - curr_left
                left = curr_left
                right = i

    return max_recovery

In [33]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [34]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [35]:
all_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(all_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(all_metrics.columns), axis=1, inplace=True)

all_metrics = all_metrics.append(rec)
all_metrics = all_metrics.append(drawd)

all_metrics = all_metrics.T
all_metrics['Method'] = ['Sharp']*len(all_metrics)

## Метрики Markov

## Поиск метрик

In [36]:
df_all_ports_pct = pd.read_csv('results/df_res_markov.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [37]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей

## Риск портфеля

In [38]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [39]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [40]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta_ports = (cov_ports['sp500']/var_ports['sp500']).to_frame().T.rename({'sp500':'Beta'})

## Коэффициент VaR

In [41]:
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [42]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [43]:
var = l_pr_values*(1+norm.ppf(0.001, mean_for_var, std_for_var))/l_pr_values-1
var = pd.DataFrame(data=var.reshape(1,-1), columns=mean_for_var.index, index=['VaR'])
var

,original_n,deco_pca_Kmean,deco_pca_Agglo,deco_pca_MiniB,deco_pca_Gauss,deco_pca_Spect,deco_fast_Kmean,deco_fast_Agglo,deco_fast_MiniB,deco_fast_Gauss,...,neur_lstm_Gauss,neur_lstm_Spect,orig__Kmean,orig__Agglo,orig__MiniB,orig__Gauss,orig__Spect,orig__origi,orig__rando,sp500
VaR,-0.083794,-0.082178,-0.086426,-0.080602,-0.083512,-0.084464,-0.08354,-0.080083,-0.0852,-0.088872,...,-0.047343,-0.087026,-0.082456,-0.085689,-0.082703,-0.078312,-0.082087,-0.083794,-0.082028,-0.108396


## Коэффициент Шарпа

In [44]:
std_ports = df_all_ports_pct.std()
sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [45]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [46]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [47]:
markov_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(markov_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(markov_metrics.columns), axis=1, inplace=True)

markov_metrics = markov_metrics.append(rec)
markov_metrics = markov_metrics.append(drawd)

markov_metrics = markov_metrics.T
markov_metrics['Method'] = ['Markov']*len(all_metrics)

In [48]:
all_metrics = pd.concat([all_metrics, markov_metrics])

## Метрики Tobin

## Поиск метрик

In [49]:
df_all_ports_pct = pd.read_csv('results/df_res_tobin.csv', index_col=0)
df_all_ports_pct.index = pd.to_datetime(df_all_ports_pct.index)
days = df_all_ports_pct.index

## Доходность

In [50]:
ret_ports = df_all_ports_pct.mean()#находим доходности портфелей

## Риск портфеля

In [51]:
risk_ports = df_all_ports_pct.std()

## Beta портфеля

In [52]:
var_ports = df_all_ports_pct.var()
cov_ports = df_all_ports_pct.cov()

In [53]:
# Beta = cov(ret_port;ret_IMOEX)/var(IMOEX), beta считается, только для портфелей, не для индекса
beta = cov_ports['sp500']/var_ports['sp500']
beta = beta.to_frame()
beta_ports = (cov_ports['sp500']/var_ports['sp500']).to_frame().T.rename({'sp500':'Beta'})

## Коэффициент VaR

In [54]:
mean_for_var = df_all_ports_pct.mean()
std_for_var = df_all_ports_pct.std()

In [55]:
l_pr_values = df_all_ports_pct.iloc[-1, :].values

In [56]:
var = l_pr_values*(1+norm.ppf(0.001, mean_for_var, std_for_var))/l_pr_values-1
var = pd.DataFrame(data=var.reshape(1,-1), columns=mean_for_var.index, index=['VaR'])
var

,original_n,deco_pca_Kmean,deco_pca_Agglo,deco_pca_MiniB,deco_pca_Gauss,deco_pca_Spect,deco_fast_Kmean,deco_fast_Agglo,deco_fast_MiniB,deco_fast_Gauss,...,neur_lstm_Gauss,neur_lstm_Spect,orig__Kmean,orig__Agglo,orig__MiniB,orig__Gauss,orig__Spect,orig__origi,orig__rando,sp500
VaR,-0.137228,-0.135844,-0.127698,-0.126594,-0.135011,-0.144219,-0.141307,-0.128967,-0.136049,-0.145637,...,-0.123954,-0.12884,-0.135581,-0.127363,-0.136278,-0.139065,-0.139585,-0.137228,-0.128086,-0.108396


## Коэффициент Шарпа

In [57]:
std_ports = df_all_ports_pct.std()
sharp_ports = (ret_ports - df_bond['цена'])/std_ports

## Просадка и восстановление

In [58]:
recovery = []
drawdown = []
for i in range(len(df_all_ports_pct.columns)):
    recovery.append(find_max_recovery(df_all_ports_pct.iloc[:,i]))
    drawdown.append(find_max_drawdown(df_all_ports_pct.iloc[:,i]))

In [59]:
risk = pd.DataFrame(risk_ports, columns =['Risk']).T
ret = pd.DataFrame(ret_ports, columns =['Ret']).T
sharp = pd.DataFrame(sharp_ports, columns =['Sharp']).T
rec = pd.Series(recovery)
rec = pd.DataFrame(rec, columns =['Recovery']).T
drawd = pd.Series(drawdown)
drawd = pd.DataFrame(drawd, columns =['Drawdown']).T

In [60]:
tobin_metrics = pd.concat([var,beta_ports, risk, ret, sharp])

rec.set_axis(list(tobin_metrics.columns), axis=1, inplace=True)
drawd.set_axis(list(tobin_metrics.columns), axis=1, inplace=True)

tobin_metrics = tobin_metrics.append(rec)
tobin_metrics = tobin_metrics.append(drawd)

tobin_metrics = tobin_metrics.T
tobin_metrics['Method'] = ['Tobin']*len(tobin_metrics)

In [61]:
all_metrics = pd.concat([all_metrics, tobin_metrics])

## Сохранение результата

In [62]:
all_metrics.to_excel('results/metrics.xlsx')